## Test notebook, looking to streamline some logic around token addresses and liquidity pool information such as name, decimal places, and the pools the tokens belong to

In [1]:
from datastreams.datastream import Streamer

import os
import pandas as pd
import polars as pl

# These commands enlarge the column size of the dataframe so things like 0x... are not truncated
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

In [4]:
weth_usdc_list = [
    "0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640", # usdc/weth .05%
    "0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8" #usdc/weth .3%
]

In [2]:
univ3_ds = Streamer('https://api.thegraph.com/subgraphs/name/messari/uniswap-v3-ethereum')

In [6]:
univ3_ds.queryDict.keys()

dict_keys(['tokens', 'rewardTokens', 'liquidityPoolFees', 'dexAmmProtocols', 'usageMetricsDailySnapshots', 'usageMetricsHourlySnapshots', 'financialsDailySnapshots', 'liquidityPools', 'liquidityPoolDailySnapshots', 'liquidityPoolHourlySnapshots', 'deposits', 'withdraws', 'swaps', 'accounts', 'activeAccounts', 'liquidityPoolAmounts', 'helperStores', 'tokenWhitelists', 'protocols', 'events'])

In [12]:
# call the field path stored in the univ3 schema
univ3_lp_fp = univ3_ds.queryDict['liquidityPools']

In [13]:
# define a custom query path based off of the field path
swaps_qp = univ3_lp_fp(
    first=10,
    where ={"id_in": weth_usdc_list},

)

In [19]:
univ3_lp_df = univ3_ds.runQuery(swaps_qp) # note that this data seems to save over the original queryDict 'pools' key

FIELD - liquidityPools


In [20]:
univ3_lp_df

,liquidityPools_id,liquidityPools_protocol_id,liquidityPools_name,liquidityPools_symbol,liquidityPools_outputToken_id,liquidityPools_isSingleSided,liquidityPools_createdTimestamp,liquidityPools_createdBlockNumber,liquidityPools_totalValueLockedUSD,liquidityPools_cumulativeSupplySideRevenueUSD,liquidityPools_cumulativeProtocolSideRevenueUSD,liquidityPools_cumulativeTotalRevenueUSD,liquidityPools_cumulativeVolumeUSD,liquidityPools_outputTokenSupply,liquidityPools_outputTokenPriceUSD,liquidityPools_stakedOutputTokenAmount,endpoint
0,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,0x1f98431c8ad98523631ae4a59f267346ea31f984,Uniswap V3 USD Coin/Wrapped Ether 0.05%,USD Coin/Wrapped Ether,None,False,1620250931,12376729,1.980760e+08,1.691806e+08,0.0,1.691806e+08,3.383611e+11,None,None,None,uniswap-v3-ethereum
1,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,0x1f98431c8ad98523631ae4a59f267346ea31f984,Uniswap V3 USD Coin/Wrapped Ether 0.3%,USD Coin/Wrapped Ether,None,False,1620169800,12370624,1.756591e+08,2.054916e+08,0.0,2.054916e+08,6.849719e+10,None,None,None,uniswap-v3-ethereum


In [26]:
# create a token_names list from liquidityPools_symbol values separated by / and drop duplicates
token_names = univ3_lp_df['liquidityPools_symbol'].str.split('/', expand=True).stack().unique()

In [27]:
token_names

array(['USD Coin', 'Wrapped Ether'], dtype=object)

In [28]:
univ3_ds.queryDict.keys()

dict_keys(['tokens', 'rewardTokens', 'liquidityPoolFees', 'dexAmmProtocols', 'usageMetricsDailySnapshots', 'usageMetricsHourlySnapshots', 'financialsDailySnapshots', 'liquidityPools', 'liquidityPoolDailySnapshots', 'liquidityPoolHourlySnapshots', 'deposits', 'withdraws', 'swaps', 'accounts', 'activeAccounts', 'liquidityPoolAmounts', 'helperStores', 'tokenWhitelists', 'protocols', 'events'])

In [30]:
univ3_tokens_fp = univ3_ds.queryDict['tokens']

In [40]:
univ3_tokens_qp = univ3_tokens_fp(
    first=10,
    where ={"symbol_in": token_names[0]},
)

In [41]:
univ3_ds.runQuery(univ3_tokens_qp)

FIELD - tokens


TypeError: mk_input_value(inner=NonNull(inner=Named(name_='String', kind='SCALAR'), kind='NON_NULL') kind='LIST', name='String' description=None kind='SCALAR', USD Coin)